In [1]:
import numpy as np
import pandas as pd

import scripts
import os
import subprocess
import importlib

In [4]:
importlib.reload(scripts)

<module 'scripts' from '/net/holy-nfsisilon/ifs/rc_labs/eddy_lab/users/lmerk/phage_groupII/scripts.py'>

In [5]:
data_dir = 'data/'

# Infernal subdir
infernal_dir = os.path.join(data_dir, 'infernal/')
g1_intron_hits = os.path.join(infernal_dir, 'g1_intron_millard.tblout')
g2_intron_hits = os.path.join(infernal_dir, 'g2_intron_millard.tblout')

# Genomes subdir
genomes_dir = os.path.join(data_dir, 'genomes/')
metadata_dir = os.path.join(genomes_dir, 'inphared_metadata/')
metadata_path = os.path.join(metadata_dir, '14Dec2023_data.tsv')
metadata = pd.read_csv(metadata_path, sep='\t').rename(columns={'Accession': 'target_name'})
genbank_dir = os.path.join(genomes_dir, 'groupII_millard/')
actual_genomes = pd.read_csv(f'{genomes_dir}actual_genomes.txt', header=None)[0].unique()

# Window and defense
window_dir = os.path.join(genomes_dir, 'gII_intron_5kb_windows/')
bakta_dir = os.path.join(data_dir, 'bakta_output')
pharokka_dir = os.path.join(data_dir, "pharokka_output")
defense_dir = os.path.join(data_dir, 'defensefinder/bakta_defense')

# Script output subdir
script_output = os.path.join(data_dir, 'script_output/')
genbank_out_directory = os.path.join(script_output, "updated_genomes")

In [6]:
preprocess = True

____

In [4]:
tblout = 'g2_intron_CP045701.tblout'

def extract_hit_df(filename, filt = True):
    initial = 0
    colspecs = []

    with open(filename, 'r') as infile:
        data = infile.readlines()
        for i in data[1:2]:
            widths = [len(col)+1 for col in i.split()]
            # print(i.split())

    for n in range(0, len(widths)):
        if n == len(widths)-1:
            colspecs.append((initial, -1))
        else:
            colspecs.append((initial, initial + widths[n]))
        initial +=  widths[n]

    hit_df = pd.read_fwf(filename, skipfooter=10, skiprows=2, colspecs = colspecs, names=['target_name', 'accession', 'query_name', 'accession_2', 'mdl', \
                                                        'mdl_from', 'mdl_to', 'seq_from', 'seq_to', 'strand', 'trunc', 'pass', 'gc', 'bias', \
                                                        'score', 'E-value', 'inc', 'description_of_target'])
    if filt == True:
        hit_df = hit_df.loc[hit_df['E-value'] < 0.01]
        
    # Some are on +, some -, make a stard and end to make comparing easier later
    hit_df['start'] = hit_df[['seq_from','seq_to']].min(axis=1)
    hit_df['end'] = hit_df[['seq_from','seq_to']].max(axis=1)

    hit_df['length'] = hit_df['end'] - hit_df['start']
    
    return hit_df

In [5]:
extract_hit_df(tblout)

,target_name,accession,query_name,accession_2,mdl,mdl_from,mdl_to,seq_from,seq_to,strand,...,pass,gc,bias,score,E-value,inc,description_of_target,start,end,length
0,CP045701.2,-,Intron_gpII,RF00029,cm,1,93,5977511,5977587,+,...,1,0.57,0.0,55.9,5.800000e-10,!,Pseudomonas brassicacearum strain S-1 chromoso...,5977511,5977587,76
1,CP045701.2,-,Intron_gpII,RF00029,cm,1,93,989520,989591,+,...,1,0.61,0.0,43.9,5.300000e-07,!,Pseudomonas brassicacearum strain S-1 chromoso...,989520,989591,71
2,CP045701.2,-,Intron_gpII,RF00029,cm,1,93,1217284,1217355,+,...,1,0.61,0.0,43.9,5.300000e-07,!,Pseudomonas brassicacearum strain S-1 chromoso...,1217284,1217355,71
3,CP045701.2,-,Intron_gpII,RF00029,cm,1,93,5423071,5423000,-,...,1,0.61,0.0,43.9,5.300000e-07,!,Pseudomonas brassicacearum strain S-1 chromoso...,5423000,5423071,71
4,CP045701.2,-,Intron_gpII,RF00029,cm,1,93,5900875,5900804,-,...,1,0.61,0.0,43.9,5.300000e-07,!,Pseudomonas brassicacearum strain S-1 chromoso...,5900804,5900875,71
5,CP045701.2,-,Intron_gpII,RF00029,cm,1,93,6376441,6376370,-,...,1,0.61,0.0,43.9,5.300000e-07,!,Pseudomonas brassicacearum strain S-1 chromoso...,6376370,6376441,71
6,CP045701.2,-,group-II-D1D4-1,RF01998,cm,1,84,988032,988115,+,...,1,0.62,0.0,60.7,3.300000e-10,!,Pseudomonas brassicacearum strain S-1 chromoso...,988032,988115,83
7,CP045701.2,-,group-II-D1D4-1,RF01998,cm,1,84,1215769,1215852,+,...,1,0.62,0.0,60.7,3.300000e-10,!,Pseudomonas brassicacearum strain S-1 chromoso...,1215769,1215852,83
8,CP045701.2,-,group-II-D1D4-1,RF01998,cm,1,84,5424559,5424476,-,...,1,0.62,0.0,60.7,3.300000e-10,!,Pseudomonas brassicacearum strain S-1 chromoso...,5424476,5424559,83
9,CP045701.2,-,group-II-D1D4-1,RF01998,cm,1,84,5902390,5902307,-,...,1,0.62,0.0,60.7,3.300000e-10,!,Pseudomonas brassicacearum strain S-1 chromoso...,5902307,5902390,83


In [15]:
from Bio import SeqIO, GenBank
from Bio.SeqFeature import SeqFeature, FeatureLocation

def update_genbank_only_g2(genbank_path):
    genome = genbank_path.split('.gb')[0]
    record = SeqIO.read(genbank_path, 'genbank')
    filename = f'g2_intron_{genome}.tblout'
    
    initial = 0
    colspecs = []

    with open(filename, 'r') as infile:
        data = infile.readlines()
        for i in data[1:2]:
            widths = [len(col)+1 for col in i.split()]
            # print(i.split())

    for n in range(0, len(widths)):
        if n == len(widths)-1:
            colspecs.append((initial, -1))
        else:
            colspecs.append((initial, initial + widths[n]))
        initial +=  widths[n]

    hit_df = pd.read_fwf(filename, skipfooter=10, skiprows=2, colspecs = colspecs, names=['target_name', 'accession', 'query_name', 'accession_2', 'mdl', \
                                                        'mdl_from', 'mdl_to', 'seq_from', 'seq_to', 'strand', 'trunc', 'pass', 'gc', 'bias', \
                                                        'score', 'E-value', 'inc', 'description_of_target'])
    # if filt == True:
    #     hit_df = hit_df.loc[hit_df['E-value'] < 0.01]
        
    # Some are on +, some -, make a stard and end to make comparing easier later
    hit_df['start'] = hit_df[['seq_from','seq_to']].min(axis=1)
    hit_df['end'] = hit_df[['seq_from','seq_to']].max(axis=1)

    hit_df['length'] = hit_df['end'] - hit_df['start']

    # Iterate over intron hits in the group
    for index, row in hit_df.iterrows():
        # Ensure start and end positions are in the correct order
        start = int(row['seq_from'])
        end = int(row['seq_to'])
        if start > end:
            start, end = end, start

        # Create a single feature spanning from start to end position
        feature = SeqFeature(FeatureLocation(start, end), type='ncRNA')

        # Add qualifiers to the feature
        feature.qualifiers['label'] = [row['query_name']]
        feature.qualifiers['query_name'] = [row['accession_2']]
        feature.qualifiers['E-value'] = [row['E-value']]

        if row['strand'] =='-':
            feature.strand = -1
        else:
            feature.strand = +1

        # Append the new feature to the existing features list in the GenBank record
        record.features.append(feature)

    # Save the modified GenBank record
    modified_genbank_file = f"{genome}_with_g2.gb"
    SeqIO.write(record, modified_genbank_file, 'genbank')

In [14]:
update_genbank_only_g2('CP045701.gb')

______

In [7]:
if preprocess:
    g1_hits = scripts.extract_hit_df(g1_intron_hits)
    g2_hits = scripts.extract_hit_df(g2_intron_hits)

    g1_df = pd.merge(g1_hits, metadata, on='target_name', how='left')
    g2_df = pd.merge(g2_hits, metadata, on='target_name', how='left')
    
    print('Dereplicating group I...')
    g1_hits_pass, g1_hits_fail = scripts.dereplicate_hits(g1_df)

    print('Saving group I...')
    # Save all the hits
    g1_df.sort_values('target_name').to_csv(f'{script_output}g1_df.csv', index=False)
    # Save the ones that didn't pass the filter
    g1_hits_fail.to_csv(f'{script_output}g1_hits_fail.csv', index=False)
    # Clean the passes, add the intronID, then save it
    g1_hits_pass = g1_hits_pass.sort_values(by=['target_name', 'seq_from'])
    g1_hits_pass['intronID'] = g1_hits_pass['target_name'] + '_I_' + g1_hits_pass['seq_from'].astype(str)
    column_order = ['intronID'] + [col for col in g1_hits_pass.columns if col != 'intronID']
    g1_hits_pass = g1_hits_pass[column_order]
    g1_hits_pass.to_csv(f'{script_output}g1_hits_pass.csv', index=False)
    
    
    print('Collapsing group II...')
    g2_hits_pass = scripts.collapse_hits(g2_df)
    
    print('Saving group II...')
    g2_hits_pass = g2_hits_pass.sort_values(by=['target_name', 'seq_from'])
    g2_hits_pass['intronID'] = g2_hits_pass['target_name'] + '_II_' + g2_hits_pass['seq_from'].astype(str)
    column_order = ['intronID'] + [col for col in g2_hits_pass.columns if col != 'intronID']
    g2_hits_pass = g2_hits_pass[column_order]
    g2_hits_pass.to_csv(f'{script_output}g2_hits_pass.csv', index=False)

    # Save all the hits
    g2_df = g2_df.sort_values(by=['target_name', 'seq_from'])
    g2_df['intronID'] = g2_df['target_name'] + '_II_' + g2_df['seq_from'].astype(str)
    column_order = ['intronID'] + [col for col in g2_df.columns if col != 'intronID']
    g2_df = g2_df[column_order]
    g2_df.to_csv(f'{script_output}g2_df.csv', index=False)
    print('Done!')
    
else:
    g1_df = pd.read_csv(f'{script_output}g1_df.csv')
    g2_df = pd.read_csv(f'{script_output}g2_df.csv')

    g1_hits_pass = pd.read_csv(f'{script_output}g1_hits_pass.csv')
    g2_hits_pass = pd.read_csv(f'{script_output}g2_hits_pass.csv')

Dereplicating group I...
Saving group I...
Collapsing group II...
Saving group II...
Done!


In [8]:
# This will submit jobs to slurm to hmmscan the surrounding window
# The genomes should be indexed (i.e. contain a .idx file in the dir)
# If not, you can use esl-index.sh

if preprocess:
    scripts.hmmscan_window(g2_df, genbank_dir, window_dir)

In [9]:
# Note you need to make sure all your jobs from above have finished before running this
# You also need to run the following bash scripts before running this:
# pharokka.sh in pharokka env to generate annotations
# bakta.sh in bakta env to generate annotations
# defensefinder.sh in defensefinder env to get defense

if preprocess:
    all_defense_hits = scripts.combine_defense(g2_df, bakta_dir, defense_dir)
    all_hit_df = scripts.hit_to_genome_with_domtbl(g2_df, window_dir, genbank_dir)
    all_hit_df_derep = all_hit_df.loc[all_hit_df['genome'].isin(actual_genomes)]
    all_hit_df_derep.to_csv(f'{script_output}g2_genes_top3_5kb_with_locs.csv', index=False)
    scripts.combine_genbanks(all_hit_df_derep, pharokka_dir, bakta_dir, genbank_out_directory, actual_genomes)
    scripts.update_genbank(genbank_out_directory, all_hit_df_derep, g1_hits_pass, g2_df, all_defense_hits)
else:
    all_hit_df_derep =  pd.read_csv(f'{script_output}g2_genes_top3_5kb_with_locs.csv')